# Importing Libraries 

In [43]:
import pandas as pd
import numpy as np
import sys

### Importing Data as pandas dataframe and preprocessing it

In [44]:
df = pd.read_csv(r"C:\Users\Aman-Verma\Downloads\ASP Transition Matrix and HMM\DWJ\May-Oct.csv",error_bad_lines=False)

In [45]:
df = df.drop('High',axis=1)
df = df.drop('Low',axis=1)
df = df.drop('Adj Close',axis=1)
df = df.drop('Volume',axis=1)

In [46]:
df.describe()

,Open,Close
count,106.000000,106.000000
mean,26386.472288,26385.129901
std,634.013614,632.055629
min,24830.160156,24815.039063
25%,25881.357422,25909.714844
50%,26393.145508,26434.310547
75%,26876.832031,26914.979980
max,27364.689453,27359.160156


In [47]:
df.head()

,Date,Open,Close
0,2019-05-01,26639.060547,26430.140625
1,2019-05-02,26407.150391,26307.789063
2,2019-05-03,26379.140625,26504.949219
3,2019-05-06,26160.619141,26438.480469
4,2019-05-07,26276.900391,25965.089844


### Setting up the parameters for the model 

In [48]:
df['Difference'] = df['Close'] - df['Open']

In [49]:
df.head()

,Date,Open,Close,Difference
0,2019-05-01,26639.060547,26430.140625,-208.919922
1,2019-05-02,26407.150391,26307.789063,-99.361328
2,2019-05-03,26379.140625,26504.949219,125.808594
3,2019-05-06,26160.619141,26438.480469,277.861328
4,2019-05-07,26276.900391,25965.089844,-311.810547


In [50]:
df.describe()

,Open,Close,Difference
count,106.000000,106.000000,106.000000
mean,26386.472288,26385.129901,-1.342386
std,634.013614,632.055629,172.498771
min,24830.160156,24815.039063,-555.660156
25%,25881.357422,25909.714844,-86.382324
50%,26393.145508,26434.310547,7.984375
75%,26876.832031,26914.979980,100.146484
max,27364.689453,27359.160156,391.029297


### Setting up the range  

In [52]:
criteria = [df['Difference'].between(-1400, -1.5), df['Difference'].between(-1.5, 1.5), df['Difference'].between(1.5,1900)]
values = ['Down', 'Stagnant', 'Up']

df['Movement'] = np.select(criteria, values, 'Stagnant')

In [53]:
df.head()

,Date,Open,Close,Difference,Movement
0,2019-05-01,26639.060547,26430.140625,-208.919922,Down
1,2019-05-02,26407.150391,26307.789063,-99.361328,Down
2,2019-05-03,26379.140625,26504.949219,125.808594,Up
3,2019-05-06,26160.619141,26438.480469,277.861328,Up
4,2019-05-07,26276.900391,25965.089844,-311.810547,Down


### Showing number of States  

In [54]:
df['Movement'].value_counts()

Up          58
Down        47
Stagnant     1
Name: Movement, dtype: int64

In [55]:
z = list(df['Movement'])

### Transition Probability Matrix 

In [56]:
pd.crosstab(pd.Series(z[1:],name='Tomorrow'),
            pd.Series(z[:-1],name='Today'),normalize=0)

Today,Down,Stagnant,Up
Tomorrow,,,
Down,0.413043,0.021739,0.565217
Stagnant,1.000000,0.000000,0.000000
Up,0.465517,0.000000,0.534483


In [57]:
from itertools import islice

def window(seq, n=2):
    "Sliding window width n from seq.  From old itertools recipes."""
    it = iter(seq)
    result = tuple(islice(it, n))
    if len(result) == n:
        yield result
    for elem in it:
        result = result[1:] + (elem,)
        yield result

In [58]:
import pandas as pd

pairs = pd.DataFrame(window(z), columns=['Today', 'Tomorrow'])
counts = pairs.groupby('Today')['Tomorrow'].value_counts()
probs = (counts).unstack()

### Transition Count Matrix

In [59]:
probs

Tomorrow,Down,Stagnant,Up
Today,,,
Down,19.0,1.0,27.0
Stagnant,1.0,NaN,NaN
Up,26.0,NaN,31.0


In [17]:
df['Difference'].abs().mean()

8.228157069129907

# End